In [4]:
import numpy as np
import pandas as pd
import os
from skimage import io, transform


# Sklearn
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score

# Torch
import torch
import torch.nn as nn
import torch.nn.functional as F 
from torch.utils.data import Dataset, DataLoader
from torch.autograd import Variable
from torchvision import datasets
from torchvision.transforms import ToTensor
from torch import tensor
import torch.optim as optim

# Plotting
import matplotlib.pyplot as plt
from matplotlib.lines import Line2D
import seaborn as sns

import warnings
warnings.filterwarnings('ignore')

### CNN classifier

In [52]:
class CNN(nn.Module):
    def __init__(self, activation_fn, drop_out_rate, batch_size, epochs, train_data, val_data, test_data, loss_fn, lr, optimizer):

        # Inherit from nn module
        super(CNN, self).__init__()

        # Define activation function
        self.activation_fn = activation_fn

        # Define drop ratio
        self.drop_out_rate = drop_out_rate

        # Define neural network architecture
        self.nn = nn.Sequential(
                # C1 6@28x28
                nn.Conv2d(1, 6, kernel_size=5, stride=1, padding=2),
                nn.BatchNorm2d(6),
                self.activation_fn(),
                nn.AvgPool2d(kernel_size=2, stride=2),
                
                # C2: 16@10x10
                nn.Dropout(self.drop_out_rate),
                nn.Conv2d(6, 16, kernel_size=5, stride=1, padding=0),
                nn.BatchNorm2d(16),
                self.activation_fn(),
                nn.AvgPool2d(kernel_size=2, stride=2),
                
                # Apply flattening on the output
                nn.Flatten(),
                
                # Dense part
                # L1
                nn.Dropout(self.drop_out_rate),
                nn.Linear(16 * 5 * 5, 120),
                nn.BatchNorm1d(120),
                self.activation_fn(),
                
                # L2
                nn.Dropout(self.drop_out_rate),
                nn.Linear(120, 84),
                nn.BatchNorm1d(84),
                self.activation_fn(),
                
                # L3
                nn.Dropout(self.drop_out_rate),
                nn.Linear(84, 10))
        
        # Define batch size
        self.batch_size = batch_size

        # Define numper of epochs
        self.epochs = epochs

        # Define datasets
        self.training_data = DataLoader(
            train_data, batch_size=self.batch_size, shuffle=True)
        self.validation_data = DataLoader(
            val_data, batch_size=self.batch_size, shuffle=True)
        self.test_data = DataLoader(
            test_data, batch_size=len(test_data), shuffle=True)

        # Define loss function
        self.loss_fn = loss_fn

        # Define learning rate
        self.lr = lr

        # Define optimizer
        self.optimizer = optimizer

        # Save training progress
        self.loss_history = []
        self.acc_history = []
    
    def forward(self, x):
        logits = self.nn(x)
        return logits
    
    def train_loop(self):
        size = len(self.training_data.dataset)
        for batch, (X, y) in enumerate(self.training_data):

            # Compute prediction and loss
            pred = self.forward(X)
            loss = self.loss_fn(pred, y)

            # Backpropagation
            self.optimizer.zero_grad()
            loss.backward()
            self.optimizer.step()

            if batch % 100 == 0:
                loss, current = loss.item(), batch * len(X)
                print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

    def val_loop(self):
        size = len(self.validation_data.dataset)
        num_batches = len(self.validation_data)
        test_loss, correct = 0, 0

        with torch.no_grad():
            for X, y in self.validation_data:
                pred = self.forward(X)
                test_loss += self.loss_fn(pred, y).item()
                correct += (pred.argmax(1) == y).type(torch.float).sum().item()
        
        test_loss /= num_batches
        correct /= size

        print(
            f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

        # Save it to history
        self.acc_history.append(correct)
        self.loss_history.append(test_loss)

    def fit(self):
        for t in range(self.epochs):
            print(f"Epoch {t+1}\n-------------------------------")
            self.train_loop()
            self.val_loop()
        print("Done!")

    def predict(self, x):
        logits = self.forward(x)
        softmax = nn.Softmax(dim=1)
        return softmax(logits).argmax(1)

    def test(self):
        # Get data
        X, y = next(iter(self.test_data))

        # Predict values
        y_hat = self.predict(X)

        print("Accuracy score for test data")
        print("-"*60)
        print(f"Acc: {accuracy_score(y, y_hat)*100} %")
        print()
        print("Confusion matrix for test data")
        print("-"*60)
        print(confusion_matrix(y, y_hat))

In [53]:
# Get training data
training_data = np.load('data/fashion_train.npy', mmap_mode='r')
X_train = training_data[:, :-1]
y_train = training_data[:, -1]

X_train.reshape(10000, 28, 28)

tensor_X_train = torch.Tensor(X_train)
tensor_y_train = torch.Tensor(y_train)

# Get test_data
test_data = np.load('data/fashion_test.npy', mmap_mode='r')
X_test = test_data[:, :-1]
y_test = test_data[:, -1]

X_test.reshape(5000, 28, 28)

tensor_X_test = torch.Tensor(X_test)
tensor_y_test = torch.Tensor(y_test)

In [54]:
# to convert the dataset X,Y into a subclass of torch DataSet so we can easily use dataloader
class ModelDataset(Dataset):
    def __init__(self,x,y):
        self.x = x
        self.y = y
        
    def __getitem__(self,idx):
        return self.x[idx], self.y[idx]
    
    def __len__(self):
        return len(self.x)

In [55]:
data_train = ModelDataset(tensor_X_train, tensor_y_train)
data_test = ModelDataset(tensor_X_test, tensor_y_test)

In [56]:
# Split training into training and validation
g_cpu = torch.Generator()
g_cpu.manual_seed(3)
data_train, data_val = torch.utils.data.random_split(data_train, [8000, 2000])

In [57]:
# Initialize model
input = {'activation_fn': nn.Sigmoid,
         'drop_out_rate': 0.,
         'batch_size': 100, 
         'epochs': 5, 
         'train_data': data_train, 
         'val_data': data_val, 
         'test_data': data_test, 
         'loss_fn': nn.CrossEntropyLoss, 
         'lr': 0.1, 
         'optimizer': torch.optim.Adam 
         }
model = CNN(input['activation_fn'], input['drop_out_rate'], input['batch_size'], input['epochs'], input['train_data'], input['val_data'], input['test_data'], input['loss_fn'], input['lr'], input['optimizer'])

In [58]:
model.fit()

Epoch 1
-------------------------------


RuntimeError: Expected 3D (unbatched) or 4D (batched) input to conv2d, but got input of size: [100, 784]

In [31]:
for batch, (X, y) in enumerate(model.training_data):
    print('j')


RuntimeError: DataLoader worker (pid(s) 1372) exited unexpectedly